## Elo Rating method

This is the Elo method.  The example is taken from "Who's #1" by Langville and Meyer and the results match.  Note, the data includes the entire 2009 NFL season, including all playoff games.  

Reference: https://www.gautamnarula.com/rating/

In [1]:
import math 
  
# Function to calculate the Probability 
def Probability(rating1, rating2): 
  
    return 1.0 * 1.0 / (1 + 1.0 * math.pow(10, 1.0 * (rating1 - rating2) / 400)) 
  
# Function to calculate Elo rating 
# K is a constant. 
# Player A wins over Player B.  
# tie = true if tie, false otherwise
def EloRating(Ra, Rb, K, tie): 
    
    # To calculate the Winning 
    # Probability of Player B 
    Pb = Probability(Ra, Rb) 
  
    # To calculate the Winning 
    # Probability of Player A 
    Pa = Probability(Rb, Ra) 
  
    # Updating the Elo Ratings 
    if tie:
       Ra = Ra + K * (1/2 - Pa) 
       Rb = Rb + K * (1/2 - Pb) 
    else:        
       Ra = Ra + K * (1 - Pa) 
       Rb = Rb + K * (0 - Pb) 
    
    return Ra, Rb

In [2]:
gameFilename = 'data/2008games.txt'
teamFilename = 'data/2008teams.txt'

k = 10

### Load the team names into an array

In [3]:
import pandas as pd

teamNames = pd.read_csv(teamFilename, header = None)
numTeams = len(teamNames)

### Load the games

In [4]:
# columns of games are:
#	column 0 = days since 1/1/0000
#	column 1 = date in YYYYMMDD format
#	column 2 = team1 index
#	column 3 = team1 homefield (1 = home, -1 = away, 0 = neutral)
#	column 4 = team1 score
#	column 5 = team2 index
#	column 6 = team2 homefield (1 = home, -1 = away, 0 = neutral)
#	column 7 = team2 score
games = pd.read_csv(gameFilename, header = None)
numGames = len(games)

### Create Elo ratings

In [5]:
import numpy as np

print(">>>>>>> I'm not including ties.")

eloRatings = np.zeros(numTeams)

for i in range(numGames):
    team1ID = games.loc[i, 2] - 1 # subtracting 1 since python indexes at 0
    team1Score = games.loc[i, 4]
    team2ID = games.loc[i, 5] - 1 # subtracting 1 since python indexes at 0
    team2Score = games.loc[i, 7]

    if team1Score > team2Score:
        team1Rating, team2Rating = EloRating(eloRatings[team1ID], eloRatings[team2ID], 32, False)
    elif team1Score < team2Score:
        team2Rating, team1Rating = EloRating(eloRatings[team2ID], eloRatings[team1ID], 32, False)
    else:  
        team1Rating, team2Rating = EloRating(eloRatings[team1ID], eloRatings[team2ID], 32, True)
        
    eloRatings[team1ID] = team1Rating
    eloRatings[team2ID] = team2Rating

>>>>>>> I'm not including ties.


### Sort and print the ranking of teams

In [6]:
iSort = np.argsort(-eloRatings)

print('\n\n************** ELO Rating Method **************\n')
print('===========================')
print('Rank   Rating      Team   ')
print('===========================')
for i in range(numTeams):
    print(f'{i+1:4d}   {eloRatings[iSort[i]]:.5f}  {teamNames.loc[iSort[i],1]}')

print('')   # extra carriage return



************** ELO Rating Method **************

Rank   Rating      Team   
   1   315.31884   North_Carolina
   2   312.40885   Memphis
   3   292.01711   UCLA
   4   276.73327   Kansas
   5   265.32082   Wisconsin
   6   255.68532   Tennessee
   7   245.99904   Butler
   8   240.37260   Texas
   9   235.84473   Georgetown
  10   221.22033   Drake
  11   218.72411   Xavier
  12   214.92761   Duke
  13   212.10799   Davidson
  14   210.15791   Kent
  15   205.31408   Stanford
  16   195.73885   W_Kentucky
  17   192.98968   BYU
  18   192.92053   Pittsburgh
  19   191.58408   UNLV
  20   188.07844   Louisville
  21   181.74699   Notre_Dame
  22   174.00486   Connecticut
  23   168.66207   Robert_Morris
  24   168.60739   Vanderbilt
  25   165.45729   Clemson
  26   165.06855   Gonzaga
  27   163.88194   South_Alabama
  28   158.92820   Washington_St
  29   158.92737   Michigan_St
  30   158.24043   Purdue
  31   156.54852   Indiana
  32   154.93935   Illinois_St
  33   152.97859   Co

### Calculate predictability of the method

In [7]:
numberCorrectPredictions = 0
for i in range(numGames):
    team1ID = games.loc[i, 2] - 1 
    team1Score = games.loc[i, 4]
    team2ID = games.loc[i, 5] - 1 
    team2Score = games.loc[i, 7]
    
    if team1Score > team2Score and eloRatings[team1ID] > eloRatings[team2ID]:
        numberCorrectPredictions += 1
    elif team2Score > team1Score and eloRatings[team2ID] > eloRatings[team1ID]:
        numberCorrectPredictions += 1
    elif team1Score == team2Score and eloRatings[team1ID] == eloRatings[team2ID]:
        numberCorrectPredictions += 1

print(f'Predictability: {numberCorrectPredictions/numGames*100:.2f}%') 


Predictability: 74.59%
